---
title: "Pset V"
author: "Summer Negahdar & Jenny Zhong"
format: 
  pdf:
    keep-tex: true
    include-in-header: 
      text: |
        \usepackage[margin=0.7in]{geometry}
        \usepackage{fvextra}
        \DefineVerbatimEnvironment{Highlighting}{Verbatim}{breaklines,commandchars=\\\{\}}
include-before-body:
  text: |
    \RecustomVerbatimEnvironment{verbatim}{Verbatim}{
      showspaces = false,
      showtabs = false,
      breaksymbolleft={},
      breaklines
    }
execute:
  code-fold: True

---

Partner 1: Summer Negahdar(samarneg)
Partner 2: Jenny Zhong (jzhong1)
This submission is our work alone and complies with the 30538 integrity policy.” Add your initials to indicate your agreement: **SN** **JZ**

we have been facing some bugs when scraping. it either does not show the link or the agency or when we subset it for the years 2021 onwards, it takes aways either of this columns. with that, we have decided to just submit it as it is since through trying to modify it other parts of the code broke (you can check our git history) therefore we have decided not to move forward with the mapping section as it requires data form other parts that are not performable


In [ ]:
import csv
from bs4 import BeautifulSoup
import pandas as pd
import requests
import altair
import requests
import datetime
from time import sleep
import os
from concurrent.futures import ThreadPoolExecutor

## Develop Initial scraper and crawler

### 1.

In [ ]:
| eval: false
def fetch_page_data(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch data from {url}")
        return []  # Return empty list if failed to fetch data

    soup = BeautifulSoup(response.content, 'html.parser')
    data = []
    actions = soup.find_all('li', class_='usa-card card--list pep-card--minimal mobile:grid-col-12')
    for action in actions:
        title_tag = action.find('h2', class_='usa-card__heading')
        title = title_tag.get_text(strip=True) if title_tag else 'No Title Provided'
        
        link_tag = title_tag.find('a')
        link = f"https://oig.hhs.gov{link_tag['href']}" if link_tag else 'No Link Provided'

        date_div = action.find('div', class_='font-body-sm margin-top-1')
        date_tag = date_div.find('span', class_='text-base-dark padding-right-105')
        date = date_tag.get_text(strip=True) if date_tag else 'No Date Provided'
        
        category_ul = action.find('ul', class_='display-inline add-list-reset')
        category_tag = category_ul.find('li')
        category = category_tag.get_text(strip=True) if category_tag else 'No Category Provided'

        data.append({'Title': title, 'Date': date, 'Category': category, 'Link': link})
    return data

# Base URL of the site to scrape
base_url = 'https://oig.hhs.gov/fraud/enforcement/'

# Fetch data from the first page only
first_page_data = fetch_page_data(base_url)


df = pd.DataFrame(first_page_data)

# this scraping is only done on the first page!
print(df.head(5))

### 2.


In [ ]:
| eval: false
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_agency_details(link):
    """ Fetches the agency details for a given URL """
    try:
        response = requests.get(link, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            uls = soup.find_all("ul", class_="usa-list usa-list--unstyled margin-y-2")
            for ul in uls:
                spans = ul.find_all("span", class_="padding-right-2 text-base")
                if len(spans) > 1:
                    agency = spans[1]
                    return agency.next_sibling.strip() if agency.next_sibling else 'N/A'
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {link}: {e}")
    return 'N/A'

def process_agencies(df):
    """ Process each link concurrently and fetch agency details """
    links = df['Link'].tolist()
    agencies = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_agency_details, link): link for link in links}
        for future in as_completed(futures):
            agencies.append(future.result())
    return agencies

# Assuming df is already loaded with the first page data and contains 'Link'
if 'Link' in df:
    df['Agency'] = process_agencies(df)
    print(df.head(5))

# Parse dates and check data types
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
print(df.dtypes)
print(df.head())

In [ ]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

# Check the data type to confirm
print(df.dtypes)
print(df.head())
print(len(df))

## Making the scraper dynamic

### 1.

a. 
Step 1: Set up function and input validation 
Function: dynamic_scraper(month, year), month and year are parameters. Check if year is at least 2013; if not, print a warning and exit function. 

Step 2: Initialize URL and variables
Set the base URL for the enforcement actions page. Initialize an empty list to store the data and create a datetime object, base_date_obj, to represent the starting date using month and year. Initialize end_flag as True to contol page looping. 

Step 3: Loop through pages until end condition
Use a while loop that continues as long as end_flag is True. For each page, retrieve the HTML content and parse it using BeautifulSoup to locate the list of enforcement actions. 

Step 4: Extract data from each enforcement action
For each action on the page, extract: title, data (converted to datetime for comparison), category and link. If the date is older than base_date_obj, set end_flag to False to stop the loop. For each link, visit the details page to retrieve the agency, name. 

Step 5: Pause and continue to the next page
Add a 1-second delay between pages to prevent server-side blockng. If a "next page" link exists, update the URL and continue, otherwise exit the loop. 

Step 6: Save data to CSV and print preview 
Convert the collected data to a pandas DataFrame. Save the DataFrame to a CSV file named enforcement_actions_year_month.csv and print the first few rows for verification. 

b. Based on the output, the total number of enforcement actions collected is 1534. 
Total enforcement actions collected: 1534
Earliest enforcement action scraped:
Date: 2023-01-03 00:00:00
Title: Podiatrist Pays $90,000 To Settle False Billing Allegations
Category: Criminal and Civil Actions
Agency: 
Link: https://oig.hhs.gov/fraud/enforcement/podiatrist-pays-90000-to-settle-false-billing-allegations/


In [ ]:
| eval: false
from time import sleep
from datetime import datetime
import time

def scrape_all_pages(base_url, base_date_obj):
    all_data = []
    page_number = 1
    stop_scraping = False

    while not stop_scraping:
        url = f"{base_url}?page={page_number}"
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        actions = soup.find_all('li', class_='usa-card card--list pep-card--minimal mobile:grid-col-12')

        if not actions:
            break  

        for action in actions:
            title_tag = action.find('h2', class_='usa-card__heading').find('a')
            title = title_tag.get_text(strip=True)
            link = f"https://oig.hhs.gov{title_tag['href']}"

            date_text = action.find('span', class_='text-base-dark padding-right-105').get_text(strip=True)
            date_object = datetime.strptime(date_text, "%B %d, %Y")

            if date_object < base_date_obj:
                stop_scraping = True
                break

            category = action.find('ul', class_='display-inline add-list-reset').get_text(strip=True)

            all_data.append({'Title': title, 'Date': date_text, 'Category': category, 'Link': link})

        page_number += 1
        time.sleep(1)

    return pd.DataFrame(all_data)

def fetch_agency(link):
    try:
        response = requests.get(link)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        agency_section = soup.find("ul", class_="usa-list usa-list--unstyled margin-y-2")
        if agency_section:
            for item in agency_section.find_all('li'):
                if "Agency:" in item.get_text():
                    return item.get_text(strip=True).replace("Agency: ", "")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {link}: {e}")
    return 'N/A'

def get_agencies(df):
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_index = {executor.submit(fetch_agency, row['Link']): index for index, row in df.iterrows()}
        agencies = ['N/A'] * len(df)
        for future in as_completed(future_to_index):
            index = future_to_index[future]
            agencies[index] = future.result()
    return agencies

def scrape_enforcement_actions(month, year):
    base_url = 'https://oig.hhs.gov/fraud/enforcement/'
    base_date_obj = datetime(year, month, 1)

    # Scrape all pages starting from the base date
    df = scrape_all_pages(base_url, base_date_obj)
    
    # Fetch agencies for each enforcement action
    df['Agency'] = get_agencies(df)
    
    # Define the output filename
    output_file = f"enforcement_actions_since_{year}_{month}.csv"
    df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

    return df

df = scrape_enforcement_actions(1, 2023)
if df is not None:
    print(df.head())

Checking the number of rows and the details. 

In [ ]:
| eval: false
output_file = "enforcement_actions_since_2023_1.csv" 
df = pd.read_csv(output_file)

num_enforcement_actions = len(df)
print(f"Total enforcement actions collected: {num_enforcement_actions}")

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

earliest_action = df.sort_values(by='Date').iloc[0]

print("\nEarliest enforcement action scraped:")
print(f"Date: {earliest_action['Date']}")
print(f"Title: {earliest_action['Title']}")
print(f"Category: {earliest_action['Category']}")
print(f"Agency: {earliest_action['Agency']}")
print(f"Link: {earliest_action['Link']}")

c. 

In [ ]:
| eval: false
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from time import sleep

def scrape_enforcement_actions(base_url, start_date):
    all_data = []
    page_number = 1
    stop_scraping = False

    while not stop_scraping:
        url = f"{base_url}?page={page_number}"
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all enforcement actions on the page
        actions = soup.find_all('li', class_='usa-card card--list pep-card--minimal mobile:grid-col-12')
        if not actions:
            break

        for action in actions:
            # Extract title, link, and date
            title_tag = action.find('h2', class_='usa-card__heading').find('a')
            title = title_tag.get_text(strip=True)
            link = f"https://oig.hhs.gov{title_tag['href']}"

            date_text = action.find('span', class_='text-base-dark padding-right-105').get_text(strip=True)
            date_object = datetime.strptime(date_text, "%B %d, %Y")

            # Stop scraping if the date is before the start date
            if date_object < start_date:
                stop_scraping = True
                break

            # Extract category and agency
            category = action.find('ul', class_='display-inline add-list-reset').get_text(strip=True)
            agency = fetch_agency(link)

            # Append data to list
            all_data.append({'Title': title, 'Date': date_text, 'Category': category, 'Link': link, 'Agency': agency})

        page_number += 1
        sleep(1)  # Add a delay to avoid overloading the server

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(all_data)
    output_file = "enforcement_actions_since_jan2021.csv"
    df.to_csv(output_file, index=False)
    print(f"Data saved to {output_file}")

    return df

def fetch_agency(link):
    try:
        response = requests.get(link)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        agency_section = soup.find("ul", class_="usa-list usa-list--unstyled margin-y-2")
        if agency_section:
            for item in agency_section.find_all('li'):
                if "Agency:" in item.get_text():
                    return item.get_text(strip=True).replace("Agency: ", "")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching agency from {link}: {e}")
    return 'N/A'

# Scrape all enforcement actions from January 2021 onwards
base_url = 'https://oig.hhs.gov/fraud/enforcement/'
start_date1 = datetime(2021, 1, 1)
df1 = scrape_enforcement_actions(base_url, start_date1)

# Display summary information
num_enforcement_actions = len(df)
print(f"Total enforcement actions collected: {num_enforcement_actions}")

# Convert 'Date' column to datetime format and find the earliest action
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
earliest_action = df.sort_values(by='Date').iloc[0]

print("\nEarliest enforcement action scraped:")
print(f"Date: {earliest_action['Date']}")
print(f"Title: {earliest_action['Title']}")
print(f"Category: {earliest_action['Category']}")
print(f"Agency: {earliest_action['Agency']}")
print(f"Link: {earliest_action['Link']}")

## Plot data based on scraped data (using Altair)

### 1.

In [ ]:
df_2021 = pd.read_csv("enforcement_actions_since202101.csv")

df_2021['date'] = pd.to_datetime(df_2021['date'], errors='coerce')

df_2021 = df_2021.dropna(subset=['date'])

df_2021['YearMonth'] = df_2021['date'].dt.to_period('M')
monthly_counts = df_2021.groupby('YearMonth').size().reset_index(name='ActionCount')
monthly_counts['YearMonth'] = monthly_counts['YearMonth'].dt.to_timestamp()

chart_step31 = alt.Chart(monthly_counts).mark_line(point=True).encode(
    x=alt.X('YearMonth:T', title='Date (Month-Year)'),
    y=alt.Y('ActionCount:Q', title='Number of Enforcement Actions')
).properties(
    title="Number of Enforcement Actions Over Time (Monthly Aggregation)",
    width=600,
    height=400
)

chart_step31.display()

![Number of enforcement actions over time](q31.png)

### 2.

In [ ]:
df_2021 = pd.read_csv("enforcement_actions_since202101.csv")

df_2021['date'] = pd.to_datetime(df_2021['date'], errors='coerce')
df_2021 = df_2021.dropna(subset=['date']) 

def categorize_title(title):
    title = title.lower()
    if any(word in title for word in ['health', 'healthcare', 'medicare', 'medical']):
        return 'Health Care Fraud'
    elif any(word in title for word in ['bank', 'financial', 'money']):
        return 'Financial Fraud'
    elif 'drug' in title or 'pharmacy' in title:
        return 'Drug Enforcement'
    elif any(word in title for word in ['bribe', 'corruption', 'corrupt']):
        return 'Bribery/Corruption'
    else:
        return 'Other'

df_2021['grouped_category'] = df_2021['title'].apply(categorize_title)

df_2021['YearMonth'] = df_2021['date'].dt.to_period('M')
monthly_counts = df_2021.groupby(['YearMonth', 'grouped_category']).size().reset_index(name='ActionCount')
monthly_counts['YearMonth'] = monthly_counts['YearMonth'].dt.to_timestamp()

chart32 = alt.Chart(monthly_counts).mark_line(point=True).encode(
    x=alt.X('YearMonth:T', title='Date (Month-Year)'),
    y=alt.Y('ActionCount:Q', title='Number of Enforcement Actions'),
    color='grouped_category:N'  # Different colors for each subcategory
).properties(
    title="Number of Enforcement Actions by Subcategory Over Time (Monthly Aggregation)",
    width=600,
    height=400
)

chart32.display()

![Number of enforcement actions by subcategory over time](q32.png)

## Create maps of enforcement activity

### 1.

### 2.

In [ ]:
import geopandas as gpd

print(df_2021.head())

In [ ]:
usattorney = gpd.read_file("42data.shp")

I ran into the same error as before although it was fixed and I could not debug. My apologies: CRSError: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: no database context specified)